In [43]:
import pandas as pd
import numpy as np
import os
import sys
import fcLib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

PLOT = False

# Import yearly load and outside temperature data from GridWorks
df = pd.read_excel(os.getcwd()+'/data/gridworks_yearly_data.xlsx', header=3, index_col = 0)
df.index = pd.to_datetime(df.index)
df.index.name = None

# Rename columns
renamed_columns = {
    'Outside Temp F': 'T_OA',
    'House Power Required AvgKw': 'Q_load'}
df.rename(columns=renamed_columns, inplace=True)

# Convert outside air temperature from °F to °C
df['T_OA'] = df['T_OA'].apply(lambda x: round(5/9 * (x-32),2))

# Keep only date, weather, and load
df = df[['T_OA','Q_load']]#[:1000]

print(f"Succesfully read hourly weather and load data ({len(df)} hours)")

# Split the data into X (weather) and y (load)
X_columns = ['T_OA']
y_columns = ['Q_load']
X = df[X_columns]
y = df[y_columns]

# Create training and testing data sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, random_state = 42)

# Create a dict to store prediction values for a 48-hour plot
scores = {}
data_plots = {}

# Iterate through each of the forecaster models
print("\nTrying different models...\n")
library = fcLib.forecasters(fcLib.forecaster_list)

for forecaster in library.forecasters:

    # Skip forecasters that bug
    print(f"- {forecaster['name']}")
    if forecaster['name'] == 'todt' or forecaster['name'] == 'sarimax_with_forecast': 
        print("--- skipped ---")
        continue

    # Fit the model to the training data and predict for the testing data
    model = getattr(fcLib, forecaster['fun'])(**forecaster['parameter'])
    model.fit(X_train, y_train)
    predict = model.predict(X_test)
    
    # Calculate the rmse and store it in the associated dataframe
    rmse = np.sqrt(mean_squared_error(y_test, predict))
    scores[forecaster['name']] = rmse
    #print(f"- RMSE = {rmse}\n")

    data_plots[forecaster['name']] = []

    # For 48-hour plot
    for i in range(48):    
        forecast = [[X.T_OA[i].tolist()]]
        predict = model.predict(forecast)

        if forecaster['name'] == 'gradient_boosting': 
            data_plots[f"{forecaster['name']}"].append(predict[0])
        else:
            data_plots[f"{forecaster['name']}"].append(predict)

    if PLOT:
        plt.figure(figsize=(15,5))
        plt.plot(df.Q_load[0:48].tolist(), label="Reality")
        for forecaster in library.forecasters:
            if forecaster['name'] == 'todt': continue
            if forecaster['name'] == 'sarimax_with_forecast': continue
            plt.plot(data_plots[f"{forecaster['name']}"], label=f"Prediction {forecaster['name']}", alpha=0.6, linestyle='dashed')
        plt.xlabel("Time")
        plt.ylabel("Load")
        plt.legend()
        plt.show()

# ---------------------------------------------------------------------
# Now include the confidence intervals using split conformal prediction
# ---------------------------------------------------------------------

# Seperate the features X (weather) and the variable y (load)
X = df[['T_OA']]
y = df[['Q_load']]

# Split the data into training and holdout sets of same size (8760/2 = 4380)
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, train_size = 0.5, random_state = 42)
n = len(X_holdout)
print(f"\n n = {len(X_holdout)}")

# Train the model with the training set
for forecaster in library.forecasters:

    # Use random forest
    if forecaster['name'] != 'multi_layer_perceptron': continue

    # Fit the model to the training data
    model = getattr(fcLib, forecaster['fun'])(**forecaster['parameter'])
    model.fit(X_train, y_train)
    
    # Use the holdout data to test the model and get residuals R_1,..., R_n
    trues, predicts, residuals = [], [], []
    percentage_list = [25, 50, 75]
    print("Computing residuals...")
    for i in range(len(X_holdout)):
        predict = model.predict([[X_holdout.T_OA[i].tolist()]])[0]
        true = y_holdout.Q_load.iloc[i]
        residual = np.abs(true-predict)
        trues.append(true)
        predicts.append(predict)
        residuals.append(residual)
        if int(i/n*100) in percentage_list: 
            print(f"... {round(i/n*100)}%")
            percentage_list = percentage_list[1:] if len(percentage_list)>1 else []
    print("Done.\n")

    # Sort the residuals
    residuals.sort(reverse=False)
    
    # Get the confidence interval for a new point
    new_point = [[5]]
    predict = model.predict(new_point)[0]
    alpha = 0.1
    CI_min = predict - residuals[int((1-alpha)*(n+1))-1]
    CI_max = predict + residuals[int((1-alpha)*(n+1))-1]
    print(f"The width of the CI = {round(residuals[int((1-alpha)*(n+1))-1]*2,4)}")
    print(f"The {round((1-alpha)*100,3)}% confidence interval of the prediction is: [{round(CI_min,4)}, {round(CI_max,4)}]")

# PLOT

if PLOT:

    plot_length = 50
    
    plt.figure(figsize=(14,5))
    
    for delta in [0.1]:
    
        predicted = []
        upper_bounds = []
        lower_bounds = []
        errors_index = [np.nan]*plot_length
        errors_count = 0
        
        for i in range(plot_length):    
            forecast = [[df.T_OA[i].tolist()]]
            
            predict = model.predict(forecast)[0]
            CI_min = predict - residuals[int((1-delta)*(n+1))-1]
            CI_max = predict + residuals[int((1-delta)*(n+1))-1]
            
            predicted.append(predict)
            lower_bounds.append(CI_min)
            upper_bounds.append(CI_max)
    
            if df.Q_load.iloc[i] > CI_max or df.Q_load.iloc[i] < CI_min:
                errors_index[i] = df.Q_load.iloc[i]
                errors_count += 1
    
        plt.plot(df.Q_load[0:plot_length].tolist(), color='blue', alpha=0.7, label="Real load", linestyle='dashed')  
        plt.plot(predicted, color='black', alpha=0.4, label='Predicted load')
        plt.fill_between(range(plot_length), lower_bounds, upper_bounds, color='gray', alpha=0.1, label=f'{round((1-delta)*100,1)}% confidence interval')
        plt.scatter(range(plot_length), errors_index, marker='o', color='red', label='Real load outside of CI')
    
    plt.xlabel("Time [hours]")
    plt.ylabel("Load [kWh]")
    plt.legend()
    plt.show()
    
    print(f"{errors_count} loads ({round(errors_count/plot_length*100,2)}%) are outside of the predicted {round((1-delta)*100,1)}% confidence interval")

Succesfully read hourly weather and load data (8760 hours)

Trying different models...

- extra_trees_pipeline
- random_forest_pipeline
- multi_layer_perceptron
- tuned_mlp
- tuned_Total_mlp
- tuned_Fast_mlp
- random_forest
- extra_trees
- gradient_boosting
- todt
--- skipped ---
- sarimax_with_forecast
--- skipped ---

 n = 4380
Computing residuals...
... 25%
... 50%
... 75%
Done.

The width of the CI = 0.4501
The 90.0% confidence interval of the prediction is: [3.3601, 3.8102]


In [44]:
print(scores)

{'extra_trees_pipeline': 0.20654640765743684, 'random_forest_pipeline': 0.19021814249635008, 'multi_layer_perceptron': 0.19612453240225236, 'tuned_mlp': 0.2006555530680047, 'tuned_Total_mlp': 0.19106542722586256, 'tuned_Fast_mlp': 0.19310949631771274, 'random_forest': 0.18983476034813465, 'extra_trees': 0.18983602040183564, 'gradient_boosting': 0.18988618709197216}


In [56]:
rmse = [value for key, value in scores.items()]
forecasters = [key for key, value in scores.items()]
print(forecasters[rmse.index(min(rmse))])

random_forest
